In [49]:
import pandas as pd
import pdfplumber
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
from openpyxl import load_workbook
from datetime import datetime
from difflib import SequenceMatcher


# Function to extract known cell values from a single sheet
def extract_from_single_sheet(excel_file_path, sheet_name):
    # Load the workbook
    wb = load_workbook(filename=excel_file_path, data_only=True)
    
    # Check if the sheet name exists in the workbook
    if sheet_name not in wb.sheetnames:
        return f"Sheet '{sheet_name}' not found in the workbook."

    # Access the specific sheet
    sheet = wb[sheet_name]

    # Adjusted cell references to where we expect the values to be
    cell_locations = {
        'Last Name' : 'B3',  # Adjusted from 'A3' assuming the value is below the label
        'First Name' : 'E3',
        'Name': 'E3',
        'Medicaid' : 'B7',
        'Tabs ID' :  'E7',
        'temp4': 'B4',
        'temp5': 'E5',
        'temp6': 'B5',
        'Address': 'G5',
        'Pin Code': 'G4',
        'Date of Birth' : 'G7',

        'Requesting Start-up Broker Services' : 'B10',
        'Broker Authorization':'G10',
        'Self-Hired Community Habilitation': 'C26',
        'Self-Hired Respite' : 'C27',
        'Self-Hired Supported Employment' : 'C28',
        'Support Brokerage' : 'C31',
        'Individual Goods and Services' : 'C32',
        'Live-In Caregiver' : 'C33',
        'Housing Subsidy' : 'H33',
        'Family Reimbursed Respite' : 'H34',
        'Other Than Personal Service Items' : 'H35',
        'Family Support Services and Assistive Supports' : 'H38',

        'Agency Name' : 'C48',
        'Contact Name' : 'H48',
        'temp1': 'C50',
        'Agency Address' : 'H50',
        'Agency Contact Email' : 'C51',
        'Agency Contact Phone' : 'C52',
        'Agency Corp ID' : 'H52',

        'Broker Name' :'C55',
        'temp2' : 'C56',
        'Broker Address': 'H56',
        'Broker Contact Email': 'C57',
        'Broker Agency Affiliation' : 'H58',

        'Care Manager Name' : 'C61',
        'temp3' : 'C56',
        'Care Manager Address' : 'H62',
        'Care Manager Contact Email' : 'C63',
        'Care Manager Contact Phone' : 'C64',
        'Agency Affiliation' : 'H64'

    }

    # Extract values from the specified cells
    extracted_values = {}
    address = ""
    for label, cell_ref in cell_locations.items():
        if label == 'Date of Birth':
            sheet[cell_ref].value = sheet[cell_ref].value.strftime('%m/%d/%Y')
        if label=='Name':
            sheet[cell_ref].value =extracted_values['First Name'].strip() + " " + extracted_values['Last Name'].strip()
            del extracted_values['First Name']
            del extracted_values['Last Name']

        if label == 'temp4' and sheet[cell_ref].value:
            address+=sheet[cell_ref].value
            
        elif label == 'temp5' and sheet[cell_ref].value:
            address=address + " " +sheet[cell_ref].value
            
        elif label == 'temp6' and sheet[cell_ref].value:
            address = address + " " +sheet[cell_ref].value

        elif label == 'Address':
            sheet[cell_ref].value =address + " " + str(sheet[cell_ref].value) if sheet[cell_ref].value else ""
            del extracted_values['temp4']
            del extracted_values['temp5']
            del extracted_values['temp6']
            address = ""
            

        if label == 'temp1':
            address  = sheet[cell_ref].value
        elif label == 'Agency Address':
            sheet[cell_ref].value = address + " "+ sheet[cell_ref].value
            del extracted_values['temp1']
            address = ""
            
        
        if label == 'temp2':
            address = sheet[cell_ref].value
        elif label == 'Broker Address':
            sheet[cell_ref].value = address + " " + sheet[cell_ref].value
            del extracted_values['temp2']
            address = ""
        
        if label == 'temp3':
            address = sheet[cell_ref].value
        elif label == 'Care Manager Address':
            sheet[cell_ref].value = address + " " + sheet[cell_ref].value
            del extracted_values['temp3']
            address = ""

        cell_value = sheet[cell_ref].value
        extracted_values[label] = cell_value

    return extracted_values

excel_file_path = r"/home/jellyfish/Simalarity/Grey, Meredith Budget 10.1.23 (1).xlsx"
sheet_name_of_interest = 'Demographics'  # Adjust this to the sheet you want to extract from
excel_data = extract_from_single_sheet(excel_file_path, sheet_name_of_interest)


/home/jellyfish/Simalarity/env/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/jellyfish/Simalarity/env/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [45]:
### LP
lp_text = ''
with pdfplumber.open('/home/jellyfish/Simalarity/Grey, Meredith LP 11.16.23 (1).pdf') as pdf:
    for i in range(len(pdf.pages)):
        lp_text = lp_text + '\n' +pdf.pages[i].extract_text()
lp_text = lp_text.strip()
print(lp_text)
    

Name :Meredith Grey Effective From : 11/16/2023 Version : 4.00
TABS ID: 654321 Life Plan Type : Annual Status:Finalized
Life Plan
Member Information
Name : Meredith Grey Date of Birth : 11/10/1996
Address : 456 Wallaby Way Medford, NY 11763 Phone : 631-987-6543
Medicaid : CD56789E Medicare :
Willowbrook Member : No Enrollment Date : 07/01/2018
CCO Information
Care Manager Name : Christina Yang Care Manager Email : Christina.Yang@myacany.org
Care Manager Phone : 347-888-8888 Care Manager Advance Care Alliance, 300 Motor
Address :
Parkway, Suite 105, Hauppauge, NY,
Care Manager Fax : 646-777-7777 11788
Supervisor Name : Miranda Bailey Supervisor Email : Miranda.Bailey@myacany.org
Supervisor Phone : 929-666-6666
CCO Name : Advance Care Alliance of NY CCO Address : 300 Motor Parkway Hauppauge NY
11788
Phone : ___-___-____
Emergency/After Hours 833-456-7890
Fax : ___-___-____ Phone :
Printed on 12/21/2023 11:09 AM Page 1 of 14
Name :Meredith Grey Effective From : 11/16/2023 Version : 4.00
T

In [46]:
pdf_path = "/home/jellyfish/Simalarity/Grey, Meredith SAP 11.16.23.pdf"
def ocr_pdf(pdf_path):
    all_text = ''
    with fitz.open(pdf_path) as doc:
        for page in doc:
            image_list = page.get_images(full=True)
            for image_index, img in enumerate(page.get_images(full=True)):
                # Extract the image bytes
                base_image = doc.extract_image(img[0])
                image_bytes = base_image["image"]

                # Convert the bytes to a PIL image
                image = Image.open(io.BytesIO(image_bytes))

                # Perform OCR using pytesseract
                text = pytesseract.image_to_string(image, lang='eng')

                all_text += text + '\n'
    return all_text

# Extract text from the PDF via OCR
sap_text = ocr_pdf(pdf_path)
print(sap_text)


Resource Center for Independent Living
Community Habilitation Staff Action Plan

Name of Individual: Meredith Grey
Medcaid Number (CIN#): CD56789E
Staff Action Plan Review Date: 11/16/2023

Name of Care Coordination Organization: Advanced CareAlliance

Broker Authorization Number: 20952

Section I Individual Habilitative Goals Valued Outcomes and Provider Assigned

My Goals Valued Outcomes

Meredith would like to be
more Involved in Community
Life

Meredith would like to decide her
own goals

 

Meredith wouk like to be
more involved in community
life

 

Habilitative Goals:

Staff Action including
frequency of service(s) and
support(s)

Provider Assigned
(Habilitative) Goals

 
  
 
 
 
 
 
  

ONGOING: Staff will Assist
Meredith with social
opportunities such as but not
limited to parks, beach's, movies
, restaurants, clothing stores and
food stores, Staff will aso Teach
Meredith when to use money
and to count out her bills.

ONGOING: Staff will Assist
Meredith with managing her
pers

In [48]:
import difflib

In [79]:
def is_present(name, text, threshold=0.6):
    # Tokenize text and name
    text_tokens = text.lower().split()
    name_tokens = name.lower().split()
    
    # Check if any token in name is present in text
    for name_token in name_tokens:
        for text_token in text_tokens:
            matcher = SequenceMatcher(None, name_token, text_token)
            if matcher.ratio() >= threshold:
                return True
    return False

In [80]:

for key, value in excel_data.items():
    if value:
        print(f'{key},{value} in excel')
        if is_present(str(value),lp_text):
            print(f'---> {key} is present in lp')
        else:
            print(f'{key} not present in lp')

        if is_present(str(value),sap_text):
            print(f'--> {key} is present in sap ')
        else:
            print(f'--> {key} not present in sap')

Name,Meredith Grey in excel
---> Name is present in lp
--> Name is present in sap 
Medicaid,CD56789E in excel
---> Medicaid is present in lp
--> Medicaid is present in sap 
Tabs ID,654321 in excel
---> Tabs ID is present in lp
--> Tabs ID not present in sap
Address,456 Wallaby Way New York Medford 11763 in excel
---> Address is present in lp
--> Address is present in sap 
Date of Birth,11/10/1996 in excel
---> Date of Birth is present in lp
--> Date of Birth not present in sap
Requesting Start-up Broker Services,No in excel
---> Requesting Start-up Broker Services is present in lp
--> Requesting Start-up Broker Services is present in sap 
Broker Authorization,20952 in excel
---> Broker Authorization is present in lp
--> Broker Authorization is present in sap 
Self-Hired Community Habilitation,Yes in excel
---> Self-Hired Community Habilitation is present in lp
--> Self-Hired Community Habilitation not present in sap
Self-Hired Respite,No in excel
---> Self-Hired Respite is present in l